https://www.learnopencv.com/camera-calibration-using-opencv/
https://www.fdxlabs.com/calculate-x-y-z-real-world-coordinates-from-a-single-camera-using-opencv/

In [4]:
import json

f = open("./DataSet/Simulation/LOWI/08/102.txt", "r")
data=json.loads(f.read())
Aircraft=data['Aircraft']
Runway=data['Runway']
Runway

{'L0': [392, 271],
 'R0': [442, 276],
 'L1': [392, 203],
 'R1': [410, 205],
 'Lat': "N47° 15.54'",
 'Lon': "E11° 19.94'",
 'Alt': 1906,
 'Mag': 79,
 'Length': 2000,
 'Width': 45,
 'Name': 'LOWI 08'}

In [11]:
L0= Runway['L0']
R0= Runway['R0']
L1= Runway['L1']
R1= Runway['R1']
Aircraft

{'Lat': "N47° 15.46'", 'Lon': "E11° 19.03'", 'Alt': 2190, 'Mag': 88}

In [21]:
import math
def measure(lat1, lon1, lat2, lon2):  ## generally used geo measurement function
    R = 6378.137; ## Radius of earth in KM
    dLat = lat2 * math.pi / 180 - lat1 * math.pi / 180
    dLon = lon2 * math.pi / 180 - lon1 * math.pi / 180
    a = math.sin(dLat/2) * math.sin(dLat/2) + math.cos(lat1 * math.pi / 180) * math.cos(lat2 * math.pi / 180) *math.sin(dLon/2) * math.sin(dLon/2);
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    d = R * c
    return d * 1000; ## meters

magPista=79

latr=47+15.54/60
lonr=11+19.94/60

lata=47+15.46/60
lona=11+19.03/60
dxy=measure(latr,lonr,lata,lona)
dy=measure(latr,0,lata,0)
dx=measure(0,lonr,0,lona)
print(dxy,dy,dx)

#aviao=[x, y, z]
aviao=[-dy, -dx, alt]
print ("aviao", aviao)

1155.4428408333895 148.4259877253941 1688.345610364497
aviao [-148.4259877253941, -1688.345610364497, 86.58536585365854]


In [22]:
import numpy as np
## para calibracao do comprimento da lente
#podem ser usados quaisquer dois pontos
alt=(Aircraft["Alt"]-Runway["Alt"])/3.28

#dimensoes da pista
Wid=Runway["Width"]/3.28
Len=Runway["Length"]/3.28
print("pista", Wid, Len)

posA=[Wid,Len,0]
posB= [Wid,0,0]
posC= [0,0,0]
posD= [0,Len,0]

pista 13.719512195121952 609.7560975609756


In [16]:
#calibracao com https://www.learnopencv.com/camera-calibration-using-opencv/
import cv2
objectPoints=np.array([[posA,posB,posC,posD]], dtype='float32')
imagePoints=np.array([[R0,R1,L1,L0]],dtype='float32')
imageSize=(1152,864)
retval, cameraMatrix, distCoeffs, rvecs, tvecs = cv2.calibrateCamera(objectPoints, imagePoints, imageSize, None, None)

In [34]:
print (cameraMatrix,'\n',rvecs,"\n\n", tvecs)

[[1.16406642e+03 0.00000000e+00 5.75500000e+02]
 [0.00000000e+00 7.83260392e+02 4.31500001e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]] 
 [[-1.2498372 ]
 [-0.2365383 ]
 [-0.02486664]] 

 [[-140.21546816]
 [-259.48822801]
 [ 889.48293918]]


In [30]:
#rotation matrix
R_mtx, jac = cv2.Rodrigues(np.array(rvecs).T)

cameraPosition = -np.matrix(R_mtx).T * np.matrix(tvecs)

cameraPosition

matrix([[ -4.74953323],
        [936.21932567],
        [-40.56147497]])

In [157]:
s_arr=np.array([0], dtype=np.float32)
s_describe=np.array([0,0,0,0,0,0,0,0,0,0],dtype=np.float32)
worldPoints=objectPoints[0]

#extrinsic matrix
Rt=np.column_stack((R_mtx,tvecs[0]))
#Projection matrix
P_mtx=newcam_mtx.dot(Rt)


total_points_used=len(worldPoints)
for i in range(0,total_points_used):  
    XYZ1=np.array([[worldPoints[i,0],worldPoints[i,1],worldPoints[i,2],1]], dtype=np.float32)
    XYZ1=XYZ1.T
    suv1=P_mtx.dot(XYZ1)
    s=suv1[2,0]    
    uv1=suv1/s
    s_arr=np.array([s/total_points_used+s_arr[0]], dtype=np.float32)
    s_describe[i]=s

In [150]:
s_describe

array([ 52.158913, 620.7276  , 622.25385 ,  53.685177,   0.      ,
         0.      ,   0.      ,   0.      ,   0.      ,   0.      ],
      dtype=float32)

In [117]:
u,v=L0
scaling_factor=320
newcam_mtx=cameraMatrix

inverse_newcam_mtx = np.linalg.inv(newcam_mtx)
inverse_R_mtx = np.linalg.inv(R_mtx)

uv_1=np.array([[u,v,1]], dtype=np.float32)
uv_1=uv_1.T
suv_1=scaling_factor*uv_1
xyz_c=inverse_newcam_mtx.dot(suv_1)
xyz_c=xyz_c-tvecs
XYZ=inverse_R_mtx.dot(xyz_c);XYZ

392 271


array([[[2.18495586e-02]],

       [[6.08254226e+02]],

       [[1.86600683e-01]]])